In [1]:
! dir D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib

 Volume in drive D is DATA
 Volume Serial Number is 62B3-38E3

 Directory of D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib

28.01.2022  13:12    <DIR>          .
28.01.2022  13:12    <DIR>          ..
30.07.2020  14:50        52,319,725 190321_Serum_Lipidextract_368723_01.mzML
30.07.2020  14:50        54,281,775 190321_Serum_Lipidextract_368723_02.mzML
30.07.2020  14:50        54,005,662 190321_Serum_Lipidextract_368723_03.mzML
30.07.2020  14:50        54,656,271 190321_Serum_Lipidextract_368723_06.mzML
30.07.2020  14:50        51,467,298 190321_Serum_Lipidextract_368723_07.mzML
30.07.2020  14:50        54,823,344 190321_Serum_Lipidextract_368723_08.mzML
30.07.2020  14:50        54,507,654 190321_Serum_Lipidextract_368723_09.mzML
30.07.2020  14:50        56,178,171 190321_Serum_Lipidextract_368723_10.mzML
30.07.2020  14:50        51,158,903 190322_Serum_Lipidextract_368723_11.mzML
30.07.2020  14:50        48,725,982 190322_Serum_Lipidextract_3687

In [2]:
from pathlib import Path


In [3]:
p = Path(r'D:\fork\lipidxplorer-evaluation\190731_benchmark_data_files_infos\190731_mzML_no_zlib')

In [4]:
mzmls = list(p.glob("*.mzml"))

In [5]:
mzmls

[WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_01.mzML'),
 WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_02.mzML'),
 WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_03.mzML'),
 WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_06.mzML'),
 WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_07.mzML'),
 WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_08.mzML'),
 WindowsPath('D:/fork/lipidxplorer-evaluation/190731_benchmark_data_files_infos/190731_mzML_no_zlib/190321_Serum_Lipidextract_368723_09.

In [6]:
from ms_deisotope import MSFileLoader

In [7]:
import pandas as pd
import numpy as np
def path2df(path, time_start=0, time_end=float("inf"), ms1_start=0, ms1_end=float("inf"), ms2_start=0, ms2_end=float("inf"), ):
    dfs = []
    with MSFileLoader(str(path)) as r:
        r.get_scan_by_time(time_start / 60)
        r.start_from_scan(r.get_scan_by_time(time_start / 60).id)
        for b in r:
            if not (time_start / 60 < b.precursor.scan_time < time_end / 60):
                continue
            a = b.precursor.arrays
            df = pd.DataFrame({"mz": a.mz, "inty": a.intensity, 'stem': path.stem, "scan_id": b.precursor.scan_id, "filter_string":b.precursor.annotations["filter string"], "precursor_id":np.nan, 'precursor_mz':np.nan})
            df = df[(df.mz.between(ms1_start,ms1_end )) & (df.inty > 0)]
            # df["scan_id"] = b.precursor.scan_id
            # df["filter_string"] = b.precursor.annotations["filter string"]
            # df["precursor_id"] = np.nan
            dfs.append(df)
            for p in b.products:
                if not (time_start / 60 < p.scan_time < time_end /60):
                    continue
                a = p.arrays
                df = pd.DataFrame({"mz": a.mz, "inty": a.intensity, 'stem': path.stem, "scan_id": p.scan_id, "filter_string":p.annotations["filter string"], "precursor_id":b.precursor.scan_id, 'precursor_mz':p.precursor_information.mz})
                df = df[(df.mz.between(ms2_start,ms2_end )) & df.inty > 0]
                # df["scan_id"] = p.scan_id  # TODO  make them categoriacal?
                # df["filter_string"] = p.annotations["filter string"]
                # df["precursor_id"] = b.precursor.scan_id
                dfs.append(df)
    df = pd.concat(dfs)

    return df

In [8]:
time_range = (33,1080)
ms1_mass_range = (360, 1000)
ms2_mass_range = (150, 1000)

In [18]:
spectra_dfs = (path2df(mzml,*time_range, *ms1_mass_range, *ms2_mass_range) for mzml in mzmls)

In [85]:
d = {"a":1,"b":2,'c':3}
d

{'a': 1, 'b': 2, 'c': 3}

In [86]:
d.setdefault(1)
d['f']

KeyError: 'f'

In [19]:
def agg_ms2_spectra_df(df):
    ms2_peaks = df.loc[~df.precursor_id.isna()]
    ms2_agg_peaks =ms2_peaks.groupby([ms2_peaks.precursor_mz.round(2), ms2_peaks.mz.round(2) ]).agg({'mz':['count','mean'], 'inty':['mean'], 'stem':['first']})
    ms2_agg_peaks = ms2_agg_peaks.loc[ms2_agg_peaks.mz['count'] > 1]
    ms2_agg_peaks.columns = ['_'.join(col).strip() for col in ms2_agg_peaks.columns.values]
    return ms2_agg_peaks

In [20]:
ms2_df = pd.concat((agg_ms2_spectra_df(df) for df in spectra_dfs ))

In [61]:
for idx, g_df in ms2_df.groupby(level='precursor_mz'):
    for inner_idx, inner_gdf in g_df.groupby(level='mz'):
        mass = inner_gdf.mz_mean.mean()
        dictIntensity = inner_gdf.set_index('stem_first')['inty_mean'].to_dict()
        dictIntensity = defaultdict(lambda: 0,dictIntensity) # to fill with zeroes


In [76]:
def mass_inty_generator_ms2(ms2_df, target):
    ms2_slice = ms2_df.loc[ms2_df.index.get_level_values(0) == target] #350.20
    for inner_idx, inner_gdf in ms2_slice.groupby(level='mz'):
        mass = inner_gdf.mz_mean.mean()
        dictIntensity = inner_gdf.set_index('stem_first')['inty_mean'].to_dict()
        dictIntensity = defaultdict(lambda: 0, dictIntensity) # to fill with zeroes
        yield mass, dictIntensity

In [78]:
from collections import defaultdict

In [79]:
[t for t in mass_inty_generator_ms2(ms2_df, 350.20)]

[(180.94058,
  defaultdict(<function __main__.mass_inty_generator_ms2.<locals>.<lambda>()>,
              {'190321_Serum_Lipidextract_368723_01': 552.115234375,
               '190321_Serum_Lipidextract_368723_02': 938.7030029296875,
               '190321_Serum_Lipidextract_368723_03': 766.4393920898438,
               '190321_Serum_Lipidextract_368723_06': 1480.437255859375,
               '190321_Serum_Lipidextract_368723_07': 1191.105712890625,
               '190321_Serum_Lipidextract_368723_08': 600.622314453125,
               '190321_Serum_Lipidextract_368723_09': 1112.54541015625,
               '190321_Serum_Lipidextract_368723_10': 756.5624389648438,
               '190322_Serum_Lipidextract_368723_11': 675.3189086914062,
               '190322_Serum_Lipidextract_368723_12': 570.8453369140625})),
 (180.96419,
  defaultdict(<function __main__.mass_inty_generator_ms2.<locals>.<lambda>()>,
              {'190321_Serum_Lipidextract_368723_01': 1771.410888671875,
               '

----

In [30]:
def mass_inty_generator(ms1_df):
    for idx, df in ms1_df.groupby(ms1_df.index):
        msmass = df.mz_mean.mean()
        smpl = df.set_index('stem_first')['inty_mean'].to_dict()
        smpl = defaultdict(lambda: 0,smpl) # to fill with zeroes
        yield msmass, dictIntensity
    


In [26]:
gdf_agg = gdf.groupby(gdf.mz.round(2)).agg({'mz':['count','mean'], 'inty':['mean']})
gdf_agg = gdf_agg.loc[gdf_agg.mz['count'] > 1]
gdf_agg

mz                     inty
           count        mean         mean
mz                                       
180.940002     4  180.940628   552.115234
180.960007     4  180.964142  1771.410889
181.940002     2  181.944061   579.149902
182.940002     4  182.936127  1185.272583
182.960007     4  182.956161  2432.938232
...          ...         ...          ...
350.279999     3  350.284332  7133.095215
350.290009     3  350.291107  2128.436523
350.299988     4  350.304474  2893.351562
350.339996     4  350.341522  4458.793457
350.950012     3  350.946442   548.440430

[74 rows x 3 columns]

In [5]:
df = pd.concat((path2df(mzml,*time_range, *ms1_mass_range, *ms2_mass_range) for mzml in mzmls))


NameError: name 'mzmls' is not defined

In [4]:
df

NameError: name 'df' is not defined

In [132]:
df[~df.precursor_id.isna()]

,mz,inty,stem,scan_id,filter_string,precursor_id
6,180.941071,644.460571,190321_Serum_Lipidextract_368723_01,controllerType=0 controllerNumber=1 scan=31,FTMS + p NSI Full ms2 350.2000@hcd30.00 [180.0...,controllerType=0 controllerNumber=1 scan=30
13,180.964157,1882.907959,190321_Serum_Lipidextract_368723_01,controllerType=0 controllerNumber=1 scan=31,FTMS + p NSI Full ms2 350.2000@hcd30.00 [180.0...,controllerType=0 controllerNumber=1 scan=30
20,181.086029,497.512695,190321_Serum_Lipidextract_368723_01,controllerType=0 controllerNumber=1 scan=31,FTMS + p NSI Full ms2 350.2000@hcd30.00 [180.0...,controllerType=0 controllerNumber=1 scan=30
27,181.943771,663.825073,190321_Serum_Lipidextract_368723_01,controllerType=0 controllerNumber=1 scan=31,FTMS + p NSI Full ms2 350.2000@hcd30.00 [180.0...,controllerType=0 controllerNumber=1 scan=30
34,182.936142,1662.708984,190321_Serum_Lipidextract_368723_01,controllerType=0 controllerNumber=1 scan=31,FTMS + p NSI Full ms2 350.2000@hcd30.00 [180.0...,controllerType=0 controllerNumber=1 scan=30
...,...,...,...,...,...,...
449,721.477478,828.841187,190322_Serum_Lipidextract_368723_12,controllerType=0 controllerNumber=1 scan=3061,FTMS + p NSI Full ms2 1036.8860@hcd30.00 [180....,controllerType=0 controllerNumber=1 scan=3060
456,747.482422,893.437256,190322_Serum_Lipidextract_368723_12,controllerType=0 controllerNumber=1 scan=3061,FTMS + p NSI Full ms2 1036.8860@hcd30.00 [180....,controllerType=0 controllerNumber=1 scan=3060
463,776.789612,1196.256836,190322_Serum_Lipidextract_368723_12,controllerType=0 controllerNumber=1 scan=3061,FTMS + p NSI Full ms2 1036.8860@hcd30.00 [180....,controllerType=0 controllerNumber=1 scan=3060
470,780.551208,984.264282,190322_Serum_Lipidextract_368723_12,controllerType=0 controllerNumber=1 scan=3061,FTMS + p NSI Full ms2 1036.8860@hcd30.00 [180....,controllerType=0 controllerNumber=1 scan=3060


In [133]:
for id, gdf in df.groupby('stem'):
    break

In [140]:
# NOTE: LX1 misses some scans, seems and lx2 does have it see 
gdf[~gdf.precursor_id.isna()].scan_id.unique().shape

(3010,)

In [ ]:
#https://stackoverflow.com/questions/39475968/aligning-pandas-dataframes-that-dont-have-a-common-index?noredirect=1&lq=1
# https://stackoverflow.com/questions/62482609/find-nearest-index-in-one-dataframe-to-another


In [1]:

# gdf[gdf.precursor_id.isna()].groupby(gdf[gdf.precursor_id.isna()].mz.round(2)).agg({'mz':['count','mean', 'std'], 'inty':['mean', 'std']})
masterscan = 
masterscandf[df.precursor_id.isna()].groupby(df[df.precursor_id.isna()].mz.round(2)).agg({'mz':['count','mean', 'std'], 'inty':['mean', 'std']}) #.unstack()

NameError: name 'df' is not defined

In [171]:
masterscan[masterscan.mz['count'] > 1]

mz                                inty               
           count        mean       std          mean            std
mz                                                                 
360.149994    80  360.150452  0.000583  48915.468750  124881.828125
360.170013     5  360.173859  0.000241  24752.093750   11788.088867
360.179993   291  360.180969  0.000713  58851.945312   35305.199219
360.190002     3  360.186829  0.001271  18705.583984    4891.850586
360.200012     5  360.202271  0.000325  19769.365234    6319.101562
...          ...         ...       ...           ...            ...
998.609985     5  998.613770  0.000375  28111.152344    4073.022949
998.789978     7  998.788330  0.000998  36884.269531   10517.687500
999.619995     3  999.618713  0.001142  21439.835938    4868.627930
999.780029     2  999.780640  0.000216  22574.683594    1142.946899
999.789978     2  999.791992  0.001252  28759.242188    6566.453125

[10803 rows x 5 columns]

In [159]:
masterscan_ms2 = df[~df.precursor_id.isna()].groupby(df[~df.precursor_id.isna()].mz.round(2)).agg({'mz':['count','mean', 'std'], 'inty':['mean', 'std']})


In [156]:
( 6*60 ) * 30 =390
vs


360

In [158]:
masterscan_ms2

mz                               inty             
           count        mean       std         mean          std
mz                                                              
180.000000    26  180.003250  0.001599   766.395691   694.520020
180.009995    33  180.009918  0.002769   603.774597   289.836151
180.020004    53  180.017487  0.001965  1189.655273  1071.578369
180.029999    21  180.030518  0.003137   584.849121   214.425446
180.039993    33  180.039444  0.003450   706.465576   573.369873
...          ...         ...       ...          ...          ...
999.859985     5  999.860168  0.000295  1730.883545  1088.960571
999.900024     1  999.898560       NaN   755.832520          NaN
999.940002    26  999.943604  0.000889  7062.706543  1970.445190
999.950012    14  999.946594  0.002074  6629.553223  1410.974365
999.989990     1  999.988037       NaN   757.195679          NaN

[72097 rows x 5 columns]

In [169]:
masterscan_ms2[masterscan_ms2.mz['count'] > masterscan_ms2.mz['count'].median()]

mz                               inty             
           count        mean       std         mean          std
mz                                                              
180.000000    26  180.003250  0.001599   766.395691   694.520020
180.009995    33  180.009918  0.002769   603.774597   289.836151
180.020004    53  180.017487  0.001965  1189.655273  1071.578369
180.029999    21  180.030518  0.003137   584.849121   214.425446
180.039993    33  180.039444  0.003450   706.465576   573.369873
...          ...         ...       ...          ...          ...
999.539978    14  999.538208  0.000821  2564.602295  1900.346436
999.640015    12  999.638733  0.003657  1073.028442   127.618729
999.849976    29  999.852905  0.001511  1776.186646   972.065186
999.940002    26  999.943604  0.000889  7062.706543  1970.445190
999.950012    14  999.946594  0.002074  6629.553223  1410.974365

[34731 rows x 5 columns]